In [1]:
import numpy as np
import json
import zipfile

In [2]:
seed = 1234
rng = np.random.default_rng(seed)

model_name = "rosenbrock"
fname = f"{model_name}.data.json"
dim = json.load(open(fname))["D"]
num_chains = 100

In [3]:
n = 1_000_000_000
x = rng.normal(1, 1, n)
y = rng.normal(x ** 2, 0.1)

In [4]:
x_mean, x_std = np.mean(x), np.std(x)
y_mean, y_std = np.mean(y), np.std(y)

x2, y2 = np.square(x), np.square(y)
x2_mean, x2_std = np.mean(x2), np.std(x2)
y2_mean, y2_std = np.mean(y2), np.std(y2)

In [5]:
print(f"{'arr':<10}\t{'mean':<10}\t\t    {'std':<10}")
print(f"x\t{x_mean}\t\t{x_std}")
print(f"y\t{y_mean}\t\t{y_std}")
print(f"x^2\t{x2_mean}\t\t{x2_std}")
print(f"y^2\t{y2_mean}\t\t{y2_std}")

arr       	mean      		    std       
x	1.0000358987042484		1.0000038750297524
y	2.0000769318777656		2.451618334525328
x^2	2.000079548771744		2.4495738604564026
y^2	10.01074019161031		25.776394778841293


# Analytic Params

In [6]:
# override emperical mean and std for analytically known values
x_mean = 1
x_std = 1

In [7]:
params_mean = {f"x[{i}]": x_mean for i in range(1, dim + 1)} | {f"y[{i}]": y_mean for i in range(1, dim + 1)}
params_std = {f"x[{i}]": x_std for i in range(1, dim + 1)} | {f"y[{i}]": y_std for i in range(1, dim + 1)}
params_squared_mean = {f"x[{i}]": x2_mean for i in range(1, dim + 1)} | {f"y[{i}]": y2_mean for i in range(1, dim + 1)}
params_squared_std = {f"x[{i}]": x2_std for i in range(1, dim + 1)} | {f"y[{i}]": y2_std for i in range(1, dim + 1)}


In [8]:
params = {
    "params_mean": params_mean,
    "params_std": params_std,
    "params_squared_mean": params_squared_mean,
    "params_squared_std": params_squared_std
}

with open("rosenbrock.analytic_params.json", "w") as f:
    json.dump(params, f, indent=4)

# Reference Draws (for init)

In [9]:
num_samples = 1000
param_names = [f"x[{i}]" for i in range(1, dim + 1)] + [f"y[{i}]" for i in range(1, dim + 1)]

x = rng.normal(1, 1, size=(num_chains, num_samples, dim))
y = rng.normal(x ** 2, 0.1)
ref_draws_np = np.concatenate([x, y], axis=2) # [num_chains, num_samples, num_params]

In [10]:
ref_draws = []
for chain in ref_draws_np:  # each chain is [n_samples, params_dim]
    chain_dict = {}
    for name, params in zip(param_names, chain.T):
        chain_dict[name] = params.tolist()
    ref_draws.append(chain_dict)

# save to file
with zipfile.ZipFile(f"{model_name}.ref_draws.json.zip", mode="w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip_file: 
    dumped_JSON: str = json.dumps(ref_draws, ensure_ascii=False, indent=4)
    zip_file.writestr(f"{model_name}.ref_draws.json", data=dumped_JSON)